In [4]:
import names
import csv
import random
import pandas as pd

num = 40
n_files = 2

TableR1_1 = csv.DictWriter(open('r1.csv', 'w+', newline=''), fieldnames=['ID', 'FirstName'])
TableR1_1.writeheader()
TableR2_1 = csv.DictWriter(open('r2.csv', 'w+', newline=''), fieldnames=['ID', 'FirstName'])
TableR2_1.writeheader()

TableS1_1 = csv.DictWriter(open('s1.csv', 'w+', newline=''), fieldnames=['ID', 'LastName'])
TableS1_1.writeheader()
TableS2_1 = csv.DictWriter(open('s2.csv', 'w+', newline=''), fieldnames=['ID', 'LastName'])
TableS2_1.writeheader()

for i in range(0,num):
    
    name = names.get_first_name()
    surname = names.get_last_name()
    
    r_loc = random.randrange(2)
    if r_loc == 1:
        TableR1_1.writerow({'ID': i, 'FirstName': name})
    else: 
        TableR2_1.writerow({'ID': i, 'FirstName': name})
    s_loc = random.randrange(2)
    if s_loc == 1:
        TableS1_1.writerow({'ID': i, 'LastName': surname})
    else: 
        TableS2_1.writerow({'ID': i, 'LastName': surname})
        
print('Wrote %d name and surname combinations on %d nodes' %(num, n_files))

Wrote 40 name and surname combinations on 2 nodes


In [6]:
# 2-phase Track-Join
# Initialize Table Tr|s

TableR1 = pd.read_csv("r1.csv")
TableS1 = pd.read_csv("s1.csv")
TableR2 = pd.read_csv("r2.csv")
TableS2 = pd.read_csv("s2.csv")

TableTrs1 = []
TableTrs2 = []

sent_to_R1 = []
sent_to_R2 = []

sent_to_S1 = []
sent_to_S2 = []

def send_to_t1(key, table):
    print('%s sent %d to TRS1' %(table, key))
    TableTrs1.append([key, table])
    
def send_to_t2(key, table):
    print('%s sent %d to TRS2' %(table, key))
    TableTrs2.append([key, table])
    
def send_to_s1(key, payload):
    print('%s sent %d to S1' %(payload.values.flatten().tolist(), key))
    sent_to_S1.append([key, payload])
    
def send_to_s2(key, payload):
    print('%s sent %d to S2' %(payload.values.flatten().tolist(), key))
    sent_to_S2.append([key, payload])

def send_to_r1(row, table):
    key = row[0]
    sent_to = row[1]
    print('%s sent %d to R1' %(table, key))
    sent_to_R1.append([key, sent_to, table])

def send_to_r2(row, table):
    key = row[0]
    sent_to = row[1]
    print('%s sent %d to R2' %(table, key))
    sent_to_R2.append([key, sent_to, table])

def hashing_fun(key):
    return (int)(key/20)
    
# ->ProcessR

print(' ---> Process R1 ')

TableTr1 = {}

for key, payload in TableR1.iterrows():
    if payload['ID'] not in TableTr1:
        if hashing_fun(key) == 0:
            send_to_t1(payload['ID'], 'R2')
        else:
            send_to_t2(payload['ID'], 'R2')
        TableTr1[payload['ID']] = [payload.values.flatten().tolist()]
    else:
        TableTr1[payload['ID']] = [TableTr1[payload['ID']], payload.values.flatten().tolist()]


print(TableTr1)

print(' ---> Process R2 ')

TableTr2 = {}

for key, payload in TableR2.iterrows():
    if payload['ID'] not in TableTr2:
        if hashing_fun(key) == 0:
            send_to_t1(payload['ID'], 'R2')
        else:
            send_to_t2(payload['ID'], 'R2')
        TableTr2[payload['ID']] = [payload.values.flatten().tolist()]
    else:
        TableTr2[payload['ID']] = [TableTr2[payload['ID']], payload.values.flatten().tolist()]

print('---Barrier ProcessR----- \n ---> Process S1 ')

# ->ProcessS

TableTs1 = {}

for key, payload in TableS1.iterrows():
    if payload['ID'] not in TableTs1:
        if hashing_fun(key) == 0:
            send_to_t1(payload['ID'], 'S1')
        else:
            send_to_t2(payload['ID'], 'S1')
        TableTs1[payload['ID']] = [payload.values.flatten().tolist()]
    else:
        TableTs1[payload['ID']] = [TableTs1[payload['ID']], payload.values.flatten().tolist()]


print('---> Process S2 ')

# ->ProcessS

TableTs2 = {}

for key, payload in TableS2.iterrows():
    if payload['ID'] not in TableTs2:
        if hashing_fun(key) == 0:
            send_to_t1(payload['ID'], 'S2')
        else:
            send_to_t2(payload['ID'], 'S2')
        TableTs2[payload['ID']] = [payload.values.flatten().tolist()]
    else:
        TableTs2[payload['ID']] = [TableTs2[payload['ID']], payload.values.flatten().tolist()]


print('---Barrier Process S----- \n ---> Process T1')

# ->ProcessT after barrier

TableTrs_cpy1 = TableTrs1
print(TableTrs1)
for row in TableTrs1:
    if row[1] == 'R1':
        for row2 in TableTrs_cpy1:
            if row2[1][0] == 'S' and row[0] == row2[0]:
                send_to_r1(row2, 'T1')
    if row[1] == 'R2':
        for row2 in TableTrs_cpy1:
            if row2[1][0] == 'S' and row[0] == row2[0]:
                send_to_r2(row2, 'T1')

print('---> Process T2')
TableTrs_cpy2 = TableTrs2

for row in TableTrs2:
    if row[1] == 'R1':
        for row2 in TableTrs_cpy2:
            if row2[1][0] == 'S' and row[0] == row2[0]:
                send_to_r1(row2, 'T2')
    if row[1] == 'R2':
        for row2 in TableTrs_cpy2:
            if row2[1][0] == 'S' and row[0] == row2[0]:
                send_to_r2(row2, 'T2')

print('---Ended Process T----- \n ---> Process R resumes')

# ->ProcessR after barrier

for row in sent_to_R1:
    for key, row_val in TableR1[TableR1['ID'] == row[0]].iterrows():
        if row[1] == 'S1':
            send_to_s1(row[0], row_val)
        if row[1] == 'S2':
            send_to_s2(row[0], row_val)
        
for row in sent_to_R2:
    for key, row_val in TableR2[TableR2['ID'] == row[0]].iterrows():
        if row[1] == 'S1':
            send_to_s1(row[0], row_val)
        if row[1] == 'S2':
            send_to_s2(row[0], row_val)

print('---Ended Process R----- \n ---> Process S1 resumes')

# ->ProcessS after barrier

for row in sent_to_S1:
    for key, row_val in TableS1[TableS1['ID'] == row[0]].iterrows():
        print(str(row_val.values.flatten().tolist()) + str(row[1].values.flatten().tolist()))

print(' -------> Processs S2')

for row in sent_to_S2:
    for key, row_val in TableS2[TableS2['ID'] == row[0]].iterrows():
        print(str(row_val.values.flatten().tolist()) + str(row[1].values.flatten().tolist()))
        
print('---Ended Process S-----')

 ---> Process R1 
R2 sent 1 to TRS1
R2 sent 6 to TRS1
R2 sent 7 to TRS1
R2 sent 11 to TRS1
R2 sent 14 to TRS1
R2 sent 15 to TRS1
R2 sent 18 to TRS1
R2 sent 21 to TRS1
R2 sent 22 to TRS1
R2 sent 24 to TRS1
R2 sent 28 to TRS1
R2 sent 30 to TRS1
R2 sent 32 to TRS1
R2 sent 33 to TRS1
R2 sent 34 to TRS1
R2 sent 36 to TRS1
{1: [[1, 'Margaret']], 6: [[6, 'Kevin']], 7: [[7, 'Mac']], 11: [[11, 'James']], 14: [[14, 'Mauricio']], 15: [[15, 'Philip']], 18: [[18, 'Brittany']], 21: [[21, 'Clara']], 22: [[22, 'Candy']], 24: [[24, 'Tillie']], 28: [[28, 'Vanessa']], 30: [[30, 'Larry']], 32: [[32, 'Cleo']], 33: [[33, 'Karen']], 34: [[34, 'Eula']], 36: [[36, 'Pearl']]}
 ---> Process R2 
R2 sent 0 to TRS1
R2 sent 2 to TRS1
R2 sent 3 to TRS1
R2 sent 4 to TRS1
R2 sent 5 to TRS1
R2 sent 8 to TRS1
R2 sent 9 to TRS1
R2 sent 10 to TRS1
R2 sent 12 to TRS1
R2 sent 13 to TRS1
R2 sent 16 to TRS1
R2 sent 17 to TRS1
R2 sent 19 to TRS1
R2 sent 20 to TRS1
R2 sent 23 to TRS1
R2 sent 25 to TRS1
R2 sent 26 to TRS1
R2 sent

In [28]:
import os
import time

timeout = time.time() + 1
counter = 0;

while True:
    stream = os.popen('echo %d' %(counter))
    if time.time() > timeout:
        print('End reached did %d calls ' %(counter))
        break
    counter = counter + 1


End reached did 320 calls 


In [31]:
for i in range(1,20):
    print(random.randrange(5))

1
1
0
4
1
0
3
2
0
4
1
2
2
0
1
2
2
2
1
